Aula 3 - Criando e manipulando tabela de símbolos (Colab)

Célula 1 — Base: Tabela de Símbolos (com escopos em pilha) + mensagens

In [1]:
#@title Base: SymbolTable (pilha de escopos) + mensagens claras
from dataclasses import dataclass, field

# ---- Erro semântico com posição ----
class SemanticError(Exception):
    def __init__(self, line, col, msg):
        super().__init__(msg)
        self.line, self.col, self.msg = line, col, msg

def show_error(src: str, line: int, col: int, msg: str):
    lines = src.splitlines()
    line = max(1, min(line, len(lines)))  # proteção
    text = lines[line-1] if lines else ""
    col = max(1, min(col, len(text)+1))
    print(f"Erro semântico na linha {line}, coluna {col}: {msg}")
    print(text)
    print(" "*(col-1) + "^")

# ---- Símbolo e tabela ----
@dataclass
class Symbol:
    name: str
    typ: str            # ex.: 'int', 'bool', ...
    kind: str           # 'var', 'param', 'func'
    line: int
    col: int
    extra: dict = field(default_factory=dict)

class SymbolTable:
    def __init__(self, src: str = ""):
        self.src = src
        self.scopes = [ {} ]   # base (global)

    # Escopos
    def enter_scope(self):
        self.scopes.append({})

    def exit_scope(self):
        if len(self.scopes) == 1:
            raise RuntimeError("não é possível sair do escopo global")
        self.scopes.pop()

    # Definir e usar
    def define(self, name: str, typ: str, kind: str, line: int, col: int, **extra):
        top = self.scopes[-1]
        if name in top:
            prev = top[name]
            raise SemanticError(line, col,
                f"redeclaração de '{name}'; já declarado neste escopo (linha {prev.line}, col {prev.col})")
        top[name] = Symbol(name, typ, kind, line, col, dict(extra))

    def use(self, name: str, line: int, col: int) -> Symbol:
        for scope in reversed(self.scopes):
            if name in scope:
                return scope[name]
        raise SemanticError(line, col, f"variável '{name}' não declarada")

    # Visualização rápida
    def debug_print(self):
        print("Escopos (topo → base):")
        for i, scope in enumerate(reversed(self.scopes), 1):
            items = ", ".join(f"{s}:{sym.typ}" for s, sym in scope.items()) or "—"
            print(f"  N{i}: {items}")


Célula 2 — Exemplo guiado: declaração, escopo interno (shadowing) e uso

In [2]:
#@title Exemplo: declarações, escopos e uso de nomes (shadowing)
program = """\
int x;
{
  int x;
  x = 1;
}
x = 2;"""

print("Programa:")
print(program)
print()

st = SymbolTable(src=program)

# global: int x;
st.define(name="x", typ="int", kind="var", line=1, col=5)
print("Após 'int x;':")
st.debug_print(); print()

# bloco interno
st.enter_scope()
st.define(name="x", typ="int", kind="var", line=3, col=7)  # sombra o x global
print("Após '{ int x; }' (x interno declarado):")
st.debug_print(); print()

# uso de x dentro do bloco → encontra o interno
try:
    sym_in = st.use("x", line=4, col=3)  # x = 1;
    print(f"Uso interno de 'x' → tipo encontrado: {sym_in.typ} (linha decl.: {sym_in.line})")
except SemanticError as e:
    show_error(program, e.line, e.col, e.msg)

# sai do bloco
st.exit_scope()
print("\nApós sair do bloco:")
st.debug_print(); print()

# uso de x fora do bloco → encontra o global
try:
    sym_out = st.use("x", line=6, col=1)  # x = 2;
    print(f"Uso externo de 'x' → tipo encontrado: {sym_out.typ} (linha decl.: {sym_out.line})")
except SemanticError as e:
    show_error(program, e.line, e.col, e.msg)


Programa:
int x;
{
  int x;
  x = 1;
}
x = 2;

Após 'int x;':
Escopos (topo → base):
  N1: x:int

Após '{ int x; }' (x interno declarado):
Escopos (topo → base):
  N1: x:int
  N2: x:int

Uso interno de 'x' → tipo encontrado: int (linha decl.: 3)

Após sair do bloco:
Escopos (topo → base):
  N1: x:int

Uso externo de 'x' → tipo encontrado: int (linha decl.: 1)


Célula 3 — Erros comuns: “não declarado” e “redeclaração”

In [3]:
#@title Erros comuns com mensagens úteis
program_err = """\
int x;
y = 3;     // erro: y não declarado
int x;     // erro: redeclaração no global
"""

print("Programa com erros:")
print(program_err)

st2 = SymbolTable(src=program_err)

# int x;
st2.define("x", "int", "var", line=1, col=5)

# y = 3; (não declarado)
try:
    st2.use("y", line=2, col=1)
except SemanticError as e:
    show_error(program_err, e.line, e.col, e.msg)

# int x; (redeclaração no mesmo escopo)
try:
    st2.define("x", "int", "var", line=3, col=5)
except SemanticError as e:
    show_error(program_err, e.line, e.col, e.msg)


Programa com erros:
int x;
y = 3;     // erro: y não declarado
int x;     // erro: redeclaração no global

Erro semântico na linha 2, coluna 1: variável 'y' não declarada
y = 3;     // erro: y não declarado
^
Erro semântico na linha 3, coluna 5: redeclaração de 'x'; já declarado neste escopo (linha 1, col 5)
int x;     // erro: redeclaração no global
    ^


Aula 4 - Verificação de tipos

In [6]:
#@title Verificação de tipos (AST + Tabela de Símbolos) — código corrigido e completo
from dataclasses import dataclass, field
from typing import List, Optional

# ====== AST (nós mínimos) ======
@dataclass
class Node:
    line: int = 1
    col: int = 1
    type: str = "unknown"  # será preenchido pelo verificador

# Programa e blocos
@dataclass
class Program(Node):
    stmts: List[Node] = field(default_factory=list)

@dataclass
class Block(Node):
    stmts: List[Node] = field(default_factory=list)

# Declarações e comandos
@dataclass
class VarDecl(Node):
    typname: str = "int"     # 'int' | 'bool'
    name: str = "x"

@dataclass
class Assign(Node):
    target: "Id" = None
    expr: Node = None

@dataclass
class If(Node):
    cond: Node = None
    then: Block = None
    els: Optional[Block] = None

@dataclass
class While(Node):
    cond: Node = None
    body: Block = None

# Expressões
@dataclass
class Num(Node):
    value: int = 0

@dataclass
class Bool(Node):
    value: bool = False

@dataclass
class Id(Node):
    name: str = "x"

@dataclass
class Bin(Node):
    op: str = "+"
    left: Node = None
    right: Node = None

# ====== Tabela de símbolos (pilha de escopos) ======
@dataclass
class Symbol:
    typ: str
    line: int
    col: int

class SymbolTable:
    def __init__(self):
        self.scopes = [ {} ]  # escopo global

    def enter(self):
        self.scopes.append({})

    def exit(self):
        if len(self.scopes) == 1:
            raise RuntimeError("não pode sair do escopo global")
        self.scopes.pop()

    def define(self, name, typ, line, col, errors):
        top = self.scopes[-1]
        if name in top:
            errors.append((line, col, f"redeclaração de '{name}' neste escopo (linha {top[name].line})"))
        else:
            top[name] = Symbol(typ, line, col)

    def lookup(self, name):
        for s in reversed(self.scopes):
            if name in s:
                return s[name]
        return None

# ====== Verificador de tipos ======
class TypeChecker:
    NUM_OPS  = {"+","-","*","/"}   # int,int -> int
    BOOL_OPS = {"&&"}              # bool,bool -> bool
    CMP_OPS  = {"<","=="}          # T,T -> bool (T igual em ambos)

    def __init__(self):
        self.errors = []
        self.sym = SymbolTable()

    def error(self, node: Node, msg: str):
        self.errors.append((node.line, node.col, msg))
        node.type = "error"
        return "error"

    def check(self, prog: Program):
        self.visit_program(prog)
        return self.errors

    # ---- Visitors ----
    def visit_program(self, node: Program):
        for s in node.stmts:
            self.visit(s)

    def visit(self, node: Node):
        if isinstance(node, Program): return self.visit_program(node)
        if isinstance(node, Block):   return self.visit_block(node)
        if isinstance(node, VarDecl): return self.visit_vardecl(node)
        if isinstance(node, Assign):  return self.visit_assign(node)
        if isinstance(node, If):      return self.visit_if(node)
        if isinstance(node, While):   return self.visit_while(node)
        if isinstance(node, Num):     node.type = "int";  return "int"
        if isinstance(node, Bool):    node.type = "bool"; return "bool"
        if isinstance(node, Id):      return self.visit_id(node)
        if isinstance(node, Bin):     return self.visit_bin(node)
        return "unknown"

    def visit_block(self, node: Block):
        self.sym.enter()
        for s in node.stmts:
            self.visit(s)
        self.sym.exit()

    def visit_vardecl(self, node: VarDecl):
        if node.typname not in ("int","bool"):
            return self.error(node, f"tipo desconhecido: {node.typname}")
        self.sym.define(node.name, node.typname, node.line, node.col, self.errors)
        node.type = "void"

    def visit_assign(self, node: Assign):
        t_id  = self.visit(node.target)  # resolve id e tipo
        t_exp = self.visit(node.expr)
        if t_id == "error" or t_exp == "error":
            return self.error(node, "atribuição inválida por erro anterior")
        if t_id != t_exp:
            return self.error(node, f"tipos diferentes na atribuição: esperado {t_id}, veio {t_exp}")
        node.type = t_id
        return t_id

    def visit_if(self, node: If):
        t = self.visit(node.cond)
        if t != "bool":
            self.error(node.cond, f"condição do if deve ser bool; veio {t}")
        self.visit_block(node.then)
        if node.els:
            self.visit_block(node.els)
        node.type = "void"

    def visit_while(self, node: While):
        t = self.visit(node.cond)
        if t != "bool":
            self.error(node.cond, f"condição do while deve ser bool; veio {t}")
        self.visit_block(node.body)
        node.type = "void"

    def visit_id(self, node: Id):
        sym = self.sym.lookup(node.name)
        if not sym:
            return self.error(node, f"variável '{node.name}' não declarada")
        node.type = sym.typ
        return sym.typ

    def visit_bin(self, node: Bin):
        lt = self.visit(node.left)
        rt = self.visit(node.right)
        op = node.op

        if lt == "error" or rt == "error":
            return self.error(node, "expressão inválida por erro anterior")

        # Aritméticos: int x int -> int
        if op in self.NUM_OPS:
            if lt == "int" and rt == "int":
                node.type = "int"
                return "int"
            return self.error(node, f"operador '{op}' exige int e int; veio {lt} e {rt}")

        # Lógicos: bool x bool -> bool
        elif op in self.BOOL_OPS:
            if lt == "bool" and rt == "bool":
                node.type = "bool"
                return "bool"
            return self.error(node, f"operador '{op}' exige bool e bool; veio {lt} e {rt}")

        # Comparações: tipos iguais -> bool
        elif op in self.CMP_OPS:
            if lt == rt and lt in ("int","bool"):
                node.type = "bool"
                return "bool"
            return self.error(node, f"comparação '{op}' exige tipos iguais; veio {lt} e {rt}")

        # Operador desconhecido
        return self.error(node, f"operador desconhecido: {op}")

# ====== Programas de exemplo ======
# OK: tipos consistentes
prog_ok = Program(stmts=[
    VarDecl(typname="int",  name="x", line=1, col=5),
    Assign(
        target=Id(name="x", line=2, col=1),
        expr=Bin(
            op="+",
            left=Num(value=2, line=2, col=5),
            right=Num(value=3, line=2, col=9),
            line=2, col=7
        ),
        line=2, col=1
    ),
    VarDecl(typname="bool", name="b", line=3, col=5),
    Assign(
        target=Id(name="b", line=4, col=1),
        expr=Bin(
            op="<",
            left=Id(name="x", line=4, col=5),
            right=Num(value=10, line=4, col=9),
            line=4, col=7
        ),
        line=4, col=1
    ),
    If(
        cond=Id(name="b", line=5, col=4),
        then=Block(
            stmts=[
                Assign(
                    target=Id(name="x", line=6, col=3),
                    expr=Bin(
                        op="+",
                        left=Id(name="x", line=6, col=7),
                        right=Num(value=1, line=6, col=11),
                        line=6, col=9
                    ),
                    line=6, col=3
                )
            ],
            line=6, col=1
        ),
        els=None,
        line=5, col=1
    ),
])

# Com erros: tipos incorretos, não declarado, condição não-bool
prog_bad = Program(stmts=[
    VarDecl(typname="int",  name="x", line=1, col=5),
    VarDecl(typname="bool", name="b", line=2, col=5),
    Assign(
        target=Id(name="b", line=3, col=1),
        expr=Bin(
            op="+",
            left=Num(value=1, line=3, col=5),
            right=Id(name="x", line=3, col=9),
            line=3, col=7
        ),
        line=3, col=1
    ),  # bool = int → erro
    Assign(
        target=Id(name="y", line=4, col=1),   # y não declarado
        expr=Num(value=2, line=4, col=5),
        line=4, col=1
    ),
    If(
        cond=Num(value=0, line=5, col=4),  # condição int → erro
        then=Block(
            stmts=[
                Assign(
                    target=Id(name="x", line=6, col=3),
                    expr=Bin(
                        op="*",
                        left=Id(name="x", line=6, col=7),
                        right=Bool(value=True, line=6, col=11),  # bool em '*'
                        line=6, col=9
                    ),
                    line=6, col=3
                )
            ],
            line=6, col=1
        ),
        els=None,
        line=5, col=1
    )
])

# ====== Execução e saída ======
def run(prog, titulo):
    print("="*60)
    print(titulo)
    tc = TypeChecker()
    errs = tc.check(prog)
    if not errs:
        print("OK: sem erros de tipo.")
    else:
        for (ln, col, msg) in errs:
            print(f"[{ln}:{col}] {msg}")

run(prog_ok,  "Programa OK")
run(prog_bad, "Programa com ERROS")


Programa OK
OK: sem erros de tipo.
Programa com ERROS
[3:1] tipos diferentes na atribuição: esperado bool, veio int
[4:1] variável 'y' não declarada
[4:1] atribuição inválida por erro anterior
[5:4] condição do if deve ser bool; veio int
[6:9] operador '*' exige int e int; veio int e bool
[6:3] atribuição inválida por erro anterior


Aula 5 - Tratando erros semânticos com mensagens úteis

In [7]:
#@title Tratando erros semânticos com mensagens úteis (3C + caret) — 1 célula
from dataclasses import dataclass, field
from typing import List, Optional

# ============================ AST (mínima) ============================
@dataclass
class Node:
    line: int = 1
    col: int = 1
    type: str = "unknown"

@dataclass
class Program(Node):
    stmts: List[Node] = field(default_factory=list)

@dataclass
class Block(Node):
    stmts: List[Node] = field(default_factory=list)

@dataclass
class VarDecl(Node):
    typname: str = "int"
    name: str = "x"

@dataclass
class Assign(Node):
    target: "Id" = None
    expr: Node = None

@dataclass
class If(Node):
    cond: Node = None
    then: Block = None
    els: Optional[Block] = None

@dataclass
class Num(Node):
    value: int = 0

@dataclass
class Bool(Node):
    value: bool = False

@dataclass
class Id(Node):
    name: str = "x"

@dataclass
class Bin(Node):
    op: str = "+"
    left: Node = None
    right: Node = None

# =================== Tabela de símbolos (escopos) ====================
@dataclass
class Symbol:
    typ: str
    line: int
    col: int

class SymbolTable:
    def __init__(self): self.scopes = [ {} ]
    def enter(self): self.scopes.append({})
    def exit(self):
        if len(self.scopes)==1: raise RuntimeError("não pode sair do global")
        self.scopes.pop()
    def define(self, name, typ, line, col, errors):
        top = self.scopes[-1]
        if name in top:
            errors.append((line, col, f"redeclaração de '{name}' neste escopo (linha {top[name].line})"))
        else:
            top[name] = Symbol(typ, line, col)
    def lookup(self, name):
        for s in reversed(self.scopes):
            if name in s: return s[name]
        return None

# ====================== Verificador de tipos =========================
class TypeChecker:
    NUM_OPS  = {"+","-","*","/"}   # int,int -> int
    BOOL_OPS = {"&&"}              # bool,bool -> bool
    CMP_OPS  = {"<","=="}          # T,T -> bool (T iguais)

    def __init__(self):
        self.errors = []
        self.sym = SymbolTable()

    def error(self, node: Node, msg: str):
        self.errors.append((node.line, node.col, msg))
        node.type = "error"
        return "error"

    def check(self, prog: Program):
        self.visit_program(prog)
        return self.errors

    # Visitors
    def visit_program(self, node: Program):
        for s in node.stmts: self.visit(s)

    def visit(self, node: Node):
        if isinstance(node, Program): return self.visit_program(node)
        if isinstance(node, Block):   return self.visit_block(node)
        if isinstance(node, VarDecl): return self.visit_vardecl(node)
        if isinstance(node, Assign):  return self.visit_assign(node)
        if isinstance(node, If):      return self.visit_if(node)
        if isinstance(node, Num):     node.type="int";  return "int"
        if isinstance(node, Bool):    node.type="bool"; return "bool"
        if isinstance(node, Id):      return self.visit_id(node)
        if isinstance(node, Bin):     return self.visit_bin(node)
        return "unknown"

    def visit_block(self, node: Block):
        self.sym.enter()
        for s in node.stmts: self.visit(s)
        self.sym.exit()

    def visit_vardecl(self, node: VarDecl):
        if node.typname not in ("int","bool"):
            return self.error(node, f"tipo desconhecido: {node.typname}")
        self.sym.define(node.name, node.typname, node.line, node.col, self.errors)
        node.type = "void"

    def visit_assign(self, node: Assign):
        t_id  = self.visit(node.target)
        t_exp = self.visit(node.expr)
        if t_id == "error" or t_exp == "error":
            return self.error(node, "atribuição inválida por erro anterior")
        if t_id != t_exp:
            return self.error(node, f"tipos diferentes na atribuição: esperado {t_id}, veio {t_exp}")
        node.type = t_id
        return t_id

    def visit_if(self, node: If):
        t = self.visit(node.cond)
        if t != "bool":
            self.error(node.cond, f"condição do if deve ser bool; veio {t}")
        self.visit_block(node.then)
        if node.els: self.visit_block(node.els)
        node.type = "void"

    def visit_id(self, node: Id):
        sym = self.sym.lookup(node.name)
        if not sym:
            return self.error(node, f"variável '{node.name}' não declarada")
        node.type = sym.typ
        return sym.typ

    def visit_bin(self, node: Bin):
        lt = self.visit(node.left)
        rt = self.visit(node.right)
        op = node.op

        if lt == "error" or rt == "error":
            return self.error(node, "expressão inválida por erro anterior")

        if op in self.NUM_OPS:
            if lt=="int" and rt=="int":
                node.type="int"; return "int"
            return self.error(node, f"operador '{op}' exige int e int; veio {lt} e {rt}")

        elif op in self.BOOL_OPS:
            if lt=="bool" and rt=="bool":
                node.type="bool"; return "bool"
            return self.error(node, f"operador '{op}' exige bool e bool; veio {lt} e {rt}")

        elif op in self.CMP_OPS:
            if lt==rt and lt in ("int","bool"):
                node.type="bool"; return "bool"
            return self.error(node, f"comparação '{op}' exige tipos iguais; veio {lt} e {rt}")

        return self.error(node, f"operador desconhecido: {op}")

# ================= Impressão 3C com caret (Contexto, Causa, Correção) ================
def print_errors_3c(errors, src: str):
    lines = src.splitlines()
    for (ln, col, msg) in errors:
        # Contexto
        print(f"[{ln}:{col}] {msg}")
        # Linha do código
        code_line = lines[ln-1] if 1 <= ln <= len(lines) else ""
        print(code_line)
        # Caret
        caret_col = max(1, col)
        print(" "*(caret_col-1) + "^")
        # Correção (dica simples baseada na mensagem)
        tip = None
        if "não declarada" in msg:
            tip = "Dica: declare a variável antes de usar."
        elif "redeclaração" in msg:
            tip = "Dica: remova a segunda declaração ou renomeie."
        elif "condição do if deve ser bool" in msg or "while" in msg:
            tip = "Dica: use uma expressão booleana (ex.: x < 3 ou true)."
        elif "exige int e int" in msg:
            tip = "Dica: ajuste os dois lados para int (sem bool)."
        elif "exige bool e bool" in msg:
            tip = "Dica: ajuste os dois lados para bool."
        elif "tipos diferentes na atribuição" in msg:
            tip = "Dica: faça a expressão ter o mesmo tipo da variável."
        if tip: print(tip)
        print()

# ================= Exemplos (OK e com erros) =========================
src_ok = """int x;
bool b;
b = x < 10;
if (b) {
  x = x + 1;
}
"""

prog_ok = Program(stmts=[
    VarDecl(typname="int",  name="x", line=1, col=5),
    VarDecl(typname="bool", name="b", line=2, col=5),
    Assign(
        target=Id(name="b", line=3, col=1),
        expr=Bin(op="<",
                 left=Id(name="x", line=3, col=5),
                 right=Num(value=10, line=3, col=9),
                 line=3, col=7),
        line=3, col=1
    ),
    If(cond=Id(name="b", line=4, col=5),
       then=Block(stmts=[
           Assign(
               target=Id(name="x", line=5, col=3),
               expr=Bin(op="+",
                        left=Id(name="x", line=5, col=7),
                        right=Num(value=1, line=5, col=11),
                        line=5, col=9),
               line=5, col=3)
       ], line=5, col=1),
       els=None, line=4, col=1)
])

src_bad = """int x;
bool b;
b = 1 + x;
y = 2;
if (0) {
  x = x * true;
}
"""

prog_bad = Program(stmts=[
    VarDecl(typname="int",  name="x", line=1, col=5),
    VarDecl(typname="bool", name="b", line=2, col=5),
    Assign(
        target=Id(name="b", line=3, col=1),
        expr=Bin(op="+",
                 left=Num(value=1, line=3, col=5),
                 right=Id(name="x", line=3, col=9),
                 line=3, col=7),
        line=3, col=1
    ),
    Assign(
        target=Id(name="y", line=4, col=1),   # não declarada
        expr=Num(value=2, line=4, col=5),
        line=4, col=1
    ),
    If(
        cond=Num(value=0, line=5, col=5),     # if precisa de bool
        then=Block(stmts=[
            Assign(
                target=Id(name="x", line=6, col=3),
                expr=Bin(op="*",
                         left=Id(name="x", line=6, col=7),
                         right=Bool(value=True, line=6, col=11),
                         line=6, col=9),
                line=6, col=3)
        ], line=6, col=1),
        els=None, line=5, col=1
    )
])

# ================= Executar e imprimir =========================
def run_and_show(src, prog, titulo):
    print("="*60)
    print(titulo)
    tc = TypeChecker()
    errors = tc.check(prog)
    if not errors:
        print("OK: sem erros semânticos.\n")
    else:
        print_errors_3c(errors, src)

run_and_show(src_ok,  prog_ok,  "Exemplo OK")
run_and_show(src_bad, prog_bad, "Exemplo com ERROS (mensagens úteis)")


Exemplo OK
OK: sem erros semânticos.

Exemplo com ERROS (mensagens úteis)
[3:1] tipos diferentes na atribuição: esperado bool, veio int
b = 1 + x;
^
Dica: faça a expressão ter o mesmo tipo da variável.

[4:1] variável 'y' não declarada
y = 2;
^
Dica: declare a variável antes de usar.

[4:1] atribuição inválida por erro anterior
y = 2;
^

[5:5] condição do if deve ser bool; veio int
if (0) {
    ^
Dica: use uma expressão booleana (ex.: x < 3 ou true).

[6:9] operador '*' exige int e int; veio int e bool
  x = x * true;
        ^
Dica: ajuste os dois lados para int (sem bool).

[6:3] atribuição inválida por erro anterior
  x = x * true;
  ^

